In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# import os
# path = ""
# path = '/content/drive/Shared drives/chinh-share/nmt-v5-Faiss/'
# os.chdir(path)
# import time
# FOLDERNAME = "OPUS-OpenNMT-" + str(time.strftime("%Y%m%d-%H%M"))
# !mkdir $FOLDERNAME

# path = path + FOLDERNAME
# os.chdir(path)
# !pwd

import os
path = '/content/drive/Shared drives/chinh-share/nmt-v5-Faiss/OPUS-OpenNMT-20210412-0158'
os.chdir(path)
!pwd

/content/drive/Shared drives/chinh-share/nmt-v5-Faiss/OPUS-OpenNMT-20210412-0158


In [3]:
!nvidia-smi

Mon Apr 12 22:34:58 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## **Install libraries**

In [4]:
!pip install OpenNMT-py==1.2.0
!pip install -U scikit-learn

     |████████████████████████████████| 204kB 8.7MB/s 
     |████████████████████████████████| 51kB 5.6MB/s 
     |████████████████████████████████| 2.6MB 6.6MB/s 
     |████████████████████████████████| 61kB 7.1MB/s 
     |████████████████████████████████| 61kB 7.4MB/s 
  Created wheel for configargparse: filename=ConfigArgParse-1.4-cp37-none-any.whl size=19638 sha256=e3857eaed9bff25b9b3cf179e038a312e2d383825be3a4429f8f5f8325574d33
  Stored in directory: /root/.cache/pip/wheels/d6/61/f7/626bbd080a9f2f70015f92025e0af663c595146083f3d9aa05
Successfully built configargparse
  Found existing installation: torchtext 0.9.1
    Uninstalling torchtext-0.9.1:
      Successfully uninstalled torchtext-0.9.1
     |████████████████████████████████| 22.3MB 1.5MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [ ]:
!wget https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/dataset/opus_bert.tar.gz
!mkdir data_bin
!tar -xvf 'opus_bert.tar.gz'

--2021-04-12 01:58:49--  https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/dataset/opus_bert.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 100661014 (96M) [application/octet-stream]
Saving to: ‘opus_bert.tar.gz’

opus_bert.tar.gz    100%[===================>]  96.00M  87.0MB/s    in 1.1s    

2021-04-12 01:58:53 (87.0 MB/s) - ‘opus_bert.tar.gz’ saved [100661014/100661014]

en_train_EM_score_0.95
vi_valid
en_train_EM_0.95
en_train_EM_factor_0.85
en_train_EM_score_0.8
vi_train
en_train_EM_factor_0.8
en_train_EM_0.8
en_valid
en_train_EM_factor_0.95
en_train
en_train_EM_score_0.85
vi_test
en_train_EM_0.85
en_train_EM_score_0.9
en_test
en_train_EM_factor_0.9
en_train_EM_0.9


In [ ]:
!mkdir -p output
!onmt_preprocess -train_src 'en_train' \\
-train_tgt 'vi_train' \\
-valid_src 'en_valid' \\
-valid_tgt 'vi_valid' \\
-save_data 'output/en-vi' 

[2021-04-12 01:59:00,750 INFO] Extracting features...
[2021-04-12 01:59:00,755 INFO]  * number of source features: 0.
[2021-04-12 01:59:00,755 INFO]  * number of target features: 0.
[2021-04-12 01:59:00,755 INFO] Building `Fields` object...
[2021-04-12 01:59:00,755 INFO] Building & saving training data...
[2021-04-12 01:59:01,823 INFO] Building shard 0.
[2021-04-12 01:59:27,926 INFO]  * saving 0th train data shard to output/en-vi.train.0.pt.
[2021-04-12 01:59:42,926 INFO]  * tgt vocab size: 50004.
[2021-04-12 01:59:43,236 INFO]  * src vocab size: 50002.
[2021-04-12 01:59:43,794 INFO] Building & saving validation data...
[2021-04-12 01:59:44,301 INFO] Building shard 0.
[2021-04-12 01:59:46,215 INFO]  * saving 0th valid data shard to output/en-vi.valid.0.pt.


In [7]:
!mkdir -p model
!onmt_train -data 'output/en-vi' \\
-save_model 'model/en-vi' \\
-layers 6 -rnn_size 512 -word_vec_size 512 -transformer_ff 2048 -heads 8 \\
-encoder_type transformer -decoder_type transformer -position_encoding \\
-train_steps 30000  -max_generator_batches 2 -dropout 0.1 -batch_size 4096 \\
-batch_type tokens -normalization tokens  -accum_count 2 -optim adam -adam_beta2 0.998 \\
-decay_method noam -warmup_steps 8000 -learning_rate 2 -max_grad_norm 0 -param_init 0 \\
-param_init_glorot -label_smoothing 0.1 -valid_steps 1000 -save_checkpoint_steps 1000 \\
-report_every 1000 -world_size 1 -gpu_ranks 0

[2021-04-12 01:59:51,696 INFO]  * src vocab size = 50002
[2021-04-12 01:59:51,696 INFO]  * tgt vocab size = 50004
[2021-04-12 01:59:51,696 INFO] Building model...
[2021-04-12 01:59:59,154 INFO] NMTModel(
  (encoder): TransformerEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(50002, 512, padding_idx=1)
        )
        (pe): PositionalEncoding(
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (transformer): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiHeadedAttention(
          (linear_keys): Linear(in_features=512, out_features=512, bias=True)
          (linear_values): Linear(in_features=512, out_features=512, bias=True)
          (linear_query): Linear(in_features=512, out_features=512, bias=True)
          (softmax): Softmax(dim=-1)
          (dropout): Dropout(p=0.1, inplace=False)
          (final_linear): Linear(in_features=512, out_feat

In [8]:
!ls -al model model/

model:
total 43395360
-rw------- 1 root root 1481228159 Apr 12 04:15 en-vi_step_10000.pt
-rw------- 1 root root 1481228159 Apr 12 02:13 en-vi_step_1000.pt
-rw------- 1 root root 1481228159 Apr 12 04:28 en-vi_step_11000.pt
-rw------- 1 root root 1481228159 Apr 12 04:42 en-vi_step_12000.pt
-rw------- 1 root root 1481228159 Apr 12 04:55 en-vi_step_13000.pt
-rw------- 1 root root 1481228159 Apr 12 05:09 en-vi_step_14000.pt
-rw------- 1 root root 1481228159 Apr 12 05:22 en-vi_step_15000.pt
-rw------- 1 root root 1481228159 Apr 12 05:36 en-vi_step_16000.pt
-rw------- 1 root root 1481228159 Apr 12 05:50 en-vi_step_17000.pt
-rw------- 1 root root 1481228159 Apr 12 06:03 en-vi_step_18000.pt
-rw------- 1 root root 1481228159 Apr 12 06:17 en-vi_step_19000.pt
-rw------- 1 root root 1481228159 Apr 12 06:30 en-vi_step_20000.pt
-rw------- 1 root root 1481228159 Apr 12 02:27 en-vi_step_2000.pt
-rw------- 1 root root 1481228159 Apr 12 06:44 en-vi_step_21000.pt
-rw------- 1 root root 1481228159 Apr 12 0

In [9]:
!onmt_translate -model model/en-vi_step_30000.pt -src en_test -tgt vi_test -output predict.txt

[2021-04-12 08:46:12,244 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:209: UserWarning: An output with one or more elements was resized since it had shape [150], which does not match the required output shape [30, 5].This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at  /pytorch/aten/src/ATen/native/Resize.cpp:19.)
  torch.mul(self.topk_scores, length_penalty, out=self.topk_log_probs)
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:209: UserWarning: An output with one or more elements was resized since it had shape [50], which does not match the required output shape [10, 5].This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an ou

In [5]:
!tail vi_test

And nobody questions him, because they don't want to hear the answer because it's a lie!
Kubo?
Họ rất vui vẻ, và lúc nào cũng hát với nến.
Nghe này, anh không thể nói chuyện bây giờ được.
Vậy thì con có thể dùng trí tưởng tượng của mình.
Không hề.
Tôi đang nhìn hắn ngay lúc này đây.
Bác không để tâm chứ?
Anh nghĩ cậu ta phản ứng với thuốc?
Bị làm sao mà anh lại đi dự lễ đặt tên em bé của Cuddy chứ?


In [ ]:
!git clone https://github.com/OpenNMT/OpenNMT-py.git

In [12]:
!ls -al

total 376662
drwx------  2 root root      4096 Apr 12 01:58 data_bin
-rw-------  1 root root   3318349 Apr 12 01:48 en_test
-rw-------  1 root root  26563375 Apr 12 01:48 en_train
-rw-------  1 root root  34866334 Apr 12 01:07 en_train_EM_0.8
-rw-------  1 root root  32006284 Apr 12 01:07 en_train_EM_0.85
-rw-------  1 root root  29696900 Apr 12 01:07 en_train_EM_0.9
-rw-------  1 root root  27849828 Apr 12 01:07 en_train_EM_0.95
-rw-------  1 root root  13223858 Apr 12 01:07 en_train_EM_factor_0.8
-rw-------  1 root root  12138268 Apr 12 01:07 en_train_EM_factor_0.85
-rw-------  1 root root  11254364 Apr 12 01:07 en_train_EM_factor_0.9
-rw-------  1 root root  10539008 Apr 12 01:07 en_train_EM_factor_0.95
-rw-------  1 root root   8152745 Apr 12 01:07 en_train_EM_score_0.8
-rw-------  1 root root   8152745 Apr 12 01:07 en_train_EM_score_0.85
-rw-------  1 root root   8152745 Apr 12 01:07 en_train_EM_score_0.9
-rw-------  1 root root   8152745 Apr 12 01:07 en_train_EM_score_0.95
-rw---

In [6]:
!perl OpenNMT-py/tools/multi-bleu.perl vi_test < predict.txt

BLEU = 17.36, 42.0/23.7/14.7/10.0 (BP=0.888, ratio=0.894, hyp_len=677925, ref_len=758454)
